In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from itertools import product

In [55]:
url = "https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20countries%20and%20dependencies%20and%20their%20capitals%20in%20native%20languages%20-%20Wikipedia.html"
html = requests.get(url)
text = html.text
soup = BeautifulSoup(text, 'html.parser')
nastevni_seznam_html = soup.select("table.wikitable") #ime classa
#print(nastevni_seznam_html)

In [78]:
drzava = []
ime = []

for vrstica in soup.find_all('tr'): #po tabeli
    celica = vrstica.find_all('td')
    if len(celica) >= 2:
        drzave = celica[0].get_text(strip=True)
        imena = celica[1].get_text(strip=True)
        drzava.append(drzave)
        ime.append(imena)




glavna_mesta = pd.DataFrame({
    "drzava": pd.Series(i for i in drzava[3:248]),
    "ime": pd.Series(i for i in ime[3:248])
})

glavna_mesta['drzava'] = glavna_mesta['drzava'].str.replace(r'\[.*?\]|\d', '', regex=True).str.strip() #drzavam odstranil []

glavna_mesta.loc[glavna_mesta['drzava'] == "China(People's Republic of)", 'drzava'] = 'China'
glavna_mesta.loc[glavna_mesta['drzava'] == "Côte d'Ivoire(formerly Ivory Coast", 'drzava'] = "Côte d'Ivoire"
glavna_mesta.loc[glavna_mesta['drzava'] == "Eswatini(formerly Swaziland)", 'drzava'] = 'Eswatini'
glavna_mesta.loc[glavna_mesta['ime'] == "HagåtñaformerlyAgaña", 'ime'] = 'Hagåtña'
glavna_mesta.loc[glavna_mesta['ime'] == "Jerusalem(declared)[a]", 'ime'] = 'Jerusalem'
glavna_mesta.loc[glavna_mesta['ime'] == "Tarawa[13]", 'ime'] = 'Tarawa' 
glavna_mesta.loc[glavna_mesta['ime'] == "Brades Estate[15]", 'ime'] = 'Brades Estate'
glavna_mesta.loc[glavna_mesta['drzava'] == "Myanmar(or Burma)", 'drzava'] = 'Myanmar'
glavna_mesta.loc[glavna_mesta['ime'] == "Yaren(de facto)[16]", 'ime'] = 'Yaren'
glavna_mesta.loc[glavna_mesta['ime'] == "East Jerusalem(declared)Ramallah(administrative capital)[b]", 'ime'] = 'East Jerusalem'
glavna_mesta.loc[glavna_mesta['ime'] == "Pretoria(administrative capital),Cape Town(legislative capital),Bloemfontein, (judicial capital)", 'ime'] = 'Pretoria'
glavna_mesta.loc[glavna_mesta['drzava'] == "Taiwan(Republic of China)", 'drzava'] = 'Taiwan'
glavna_mesta.loc[glavna_mesta['ime'] == "Bern(de facto)[16]", 'ime'] = 'Bern'
glavna_mesta.loc[glavna_mesta['ime'] == "Fongafale(inFunafuti)", 'ime'] = 'Funafuti'
glavna_mesta.to_csv("glavna_mesta.csv", index=False)


In [135]:
filmi = pd.read_csv("SYB61_253_Population Growth Rates in Urban areas and Capital cities.csv")
filmi = filmi.iloc[:, [1, 2, 3, 6]] #vzeli stoplce
filmi.columns = ['enota', 'leto', 'spremenljivka', 'vrednost'] #spremenili ime
filmi = filmi.iloc[11:] #odstranili prvih 11

filmi.reset_index(drop=True, inplace=True) #resetirali indexe
filtriran = filmi[filmi.iloc[:, 2].str.contains(r"Urban population \(percent\)|Capital city population \(thousands\)|Capital city population \(as a percentage of total population\)", na=False, regex=True)] #vzeli stoplce ki so take oblike
filtriran["vrednost"] = filtriran["vrednost"].str.replace(",", "").astype(float) #string spremenil v float

populacija = filtriran.astype({'leto': int, 'spremenljivka': 'category', 'vrednost': float}) #spremenili atribute
populacija.to_csv("poplacija.csv", index=False)
#populacija.dtypes

C:\Users\blabl\AppData\Local\Temp\ipykernel_9800\3196112288.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtriran["vrednost"] = filtriran["vrednost"].str.replace(",", "").astype(float)


enota              object
leto                int64
spremenljivka    category
vrednost          float64
dtype: object

In [155]:
url = "https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20Academy%20Award%e2%80%93winning%20films%20-%20Wikipedia.html"
html = requests.get(url)
text = html.text
soup = BeautifulSoup(text, 'html.parser')


naslov = []
leto = []
oskar =[]
stevilo =[]

for vrstica in soup.find_all('tr'): #po tabeli
    celica = vrstica.find_all('td')
    if len(celica) >= 2:
        prvi = celica[0].get_text(strip=True)
        drugi = celica[1].get_text(strip=True)
        tretji = celica[2].get_text(strip=True)
        cetrti = celica[3].get_text(strip=True)
        naslov.append(prvi)
        leto.append(drugi)
        oskar.append(tretji)
        stevilo.append(cetrti)


oskar2 = []
stevilo2 = []
for text in oskar:
  oskar2.append(re.findall(r'(?<!\()\b\d+\b(?!\))', text))
for text in stevilo:
   stevilo2.append(re.findall(r'(?<!\()\b\d+\b(?!\))', text))
leto2 = [
    re.match(r'^(\d+)', year).group(1) if '/' in year else year
    for year in leto]

oskar3 = [int(item[0]) for item in oskar2]
stevilo3 = [int(item[0]) for item in stevilo2]

rez = [x - y for x, y in zip(stevilo3, oskar3)]


filmi_true = pd.DataFrame({
    "naslov": pd.Series(naslov),
    "leto": pd.Series(leto2),
    "oskar": pd.Series(),
    "stevilo": pd.Series(oskar3),
    "stevilo2": pd.Series(stevilo3)
})

filmi_true = filmi_true.fillna(True)


filmi_false = filmi_true.copy()
filmi_false['oskar'] = False
filmi_false['stevilo'] = rez

filmi = pd.concat([filmi_false, filmi_true], ignore_index=True)
filmi = filmi.sort_values(by='naslov', ascending=True)
filmi = filmi[filmi['stevilo2'] != 0]
filmi = filmi.drop(columns=['stevilo2'])
filmi = filmi.reset_index(drop=True)
filmi = filmi.astype({'leto': int})

filmi.to_csv("filmi.csv", index=False)



C:\Users\blabl\AppData\Local\Temp\ipykernel_16100\982439136.py:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filmi_true = filmi_true.fillna(True)


In [226]:
url = "https://kt.ijs.si/~ljupco/lectures/papvp-2425/podatkovni-viri/List%20of%20Academy%20Award%e2%80%93winning%20foreign-language%20films%20-%20Wikipedia.html"

tables = pd.read_html(url)
df = pd.DataFrame(tables[0])

df = df.drop(columns=['Winner(s)'])
df['leto'] = df['Year (Ceremony)'].str.extract(r'(\d{4})')
df['zap_st_podelitev'] = df['Year (Ceremony)'].str.extract(r'\((\d+)[a-z]{2}\)')
df = df.drop(columns=['Year (Ceremony)'])


df['naslov_nom'] = df['Film title used in nomination'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['naslov_org'] = df['Original title'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['kategorija'] = df['Category'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

df = df.drop(columns=['Film title used in nomination','Original title', 'Category'])
df = df[['naslov_nom','naslov_org','leto','zap_st_podelitev', 'kategorija','Country','Language(s)']]

df['drzava'] = df['Country'].str.split(', ')
df['jezik'] = df['Language(s)'].str.split(', ')

expanded_rows = []
for _, row in df.iterrows():
    for country, language in product(row['drzava'], row['jezik']):
        expanded_rows.append({
            'naslov_nom': row['naslov_nom'],
            'naslov_org': row['naslov_org'],
            'leto': row['leto'],
            'zap_st_podelitev': row['zap_st_podelitev'],
            'kategorija': row['kategorija'],
            'drzava': country,
            'jezik': language
        })
tuji_filmi = pd.DataFrame(expanded_rows)
#print(df)

